In [1]:
import sqlite3
import numpy as np
from sklearn.cluster import KMeans
import json
import plotly
import plotly.graph_objs as go
import plotly.plotly as py

In [2]:
debug = False

In [3]:
conn = sqlite3.connect('livs.db')

conn.row_factory = sqlite3.Row

curs = conn.cursor()

result = []
for row in curs.execute('select "Kolhydrater_g", "Fett_g", "Protein_g" from livs'):
    result.append(row)

result = result[1:]  # skippa första raden, den är kolumnrubriker

if debug:
    print ('result')
    print (result)

conn.close()

dataSet = np.array(result)

print (dataSet)

[[   0.     85.      2.8 ]
 [   0.    100.      0.  ]
 [   0.    100.      0.  ]
 ..., 
 [   7.6     0.5     0.93]
 [  72.6     3.55    5.63]
 [  57.5     4.8     8.44]]


In [4]:
 def cluster(dataSet):
    kmeans = KMeans(n_clusters=numClusters)
    kmeans.fit(dataSet)

    centroids = kmeans.cluster_centers_
    labels = kmeans.labels_

    if debug:
        print (centroids)
        print (labels)

    return labels

In [5]:
numClusters = 10

In [6]:
labels = cluster(dataSet)

# transpose to work with Plotly-format (don't remember if it is sklearn.cluster or plotly that is counterintuitive)
dataToChart = dataSet.transpose()

# create multi dimensional array of data by label
segmentedData = [[[] for _ in range(3)] for _ in range(numClusters)]

for num, label in enumerate(labels):
    if debug:
        print (str(num), str(label))
    segmentedData[label][0].append(dataToChart[0][num])
    segmentedData[label][1].append(dataToChart[1][num])
    segmentedData[label][2].append(dataToChart[2][num])

if debug:
    print(segmentedData)

In [7]:
#plotly.offline.init_notebook_mode(connected=True)
    
traces = []
baseColor = 100
i = 0
while i < numClusters:
    trace = go.Scatter3d(
        x=segmentedData[i][0],
        y=segmentedData[i][1],
        z=segmentedData[i][2],
        mode='markers',
        marker=dict(
            size=12,
            line=dict(
                color='rgba(baseColor+(i*2), baseColor+(i*2), baseColor+(i*2), 0.14)',
                width=0.5
            ),
            opacity=0.8
        ),
        # @todo: fix names list for plotly
        #text=names
    )
    traces.append(trace)
    i+=1

layout = go.Layout(
    scene=go.Scene(
        xaxis=go.XAxis(title='Carbs', tickprefix='Carbs ', showtickprefix='first'),
        yaxis=go.YAxis(title='Fat', tickprefix='Fat ', showtickprefix='first'),
        zaxis=go.ZAxis(title='Protein', tickprefix='Protein ', showtickprefix='first')
    ),
    height = 800,
    width = 800,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='table1')

Plotly-visualiseringen ser ut så här med 10 kluster
![caption](plotly.png)